In [ ]:
# header files
%matplotlib inline
import os
import glob
import csv
import numpy as np
import pandas as pd
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.svm import HingeLossSurvivalSVM
from sksurv.metrics import (
    concordance_index_censored,
    concordance_index_ipcw,
    cumulative_dynamic_auc,
    integrated_brier_score,
)
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression, f_classif
import matplotlib.pyplot as plt
from datetime import datetime
plt.rcParams['figure.figsize'] = [4, 4]
print("Header files loaded!")

In [ ]:
tcga_oc_files = glob.glob("survival_analysis_results/ovary/tcga/*")
upmc_oc_files = glob.glob("survival_analysis_results/ovary/upmc/*")
ucla_oc_files = glob.glob("survival_analysis_results/ovary/ucla/*")
print(len(tcga_oc_files))
print(len(upmc_oc_files))
print(len(ucla_oc_files))

In [ ]:
# create output survival information for training model
is_ovarian_cancer = 1
if is_ovarian_cancer:
    pfs_event_temp = []
    pfs_days_temp = []
    os_event_temp = []
    os_days_temp = []
    filenames_temp = []
    age_temp = []
    stage_temp = []
    hrd_temp = []
    cr_temp = []
    pr_temp = []
    pd_temp = []
    brca_temp = []
    debulking_temp = []
    flag = -1
    c = 0
    with open("survival_analysis_results/ovary/tcga.csv", newline='', encoding = "ISO-8859-1") as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                flag = 1
            else:
                array = row
                filenames_temp.append(array[0])
                age_temp.append(float(array[1]))
                os_days_temp.append(float(array[10])*30.5)
                pfs_days_temp.append(float(array[11])*30.5)
                
                if array[12] == "1:DECEASED":
                    os_event_temp.append(True)
                else:
                    os_event_temp.append(False)
                
                if array[13] == "1:PROGRESSION":
                    pfs_event_temp.append(True)
                else:
                    pfs_event_temp.append(False)
                    
                if array[17] == "Stage IIIC":
                    stage_temp.append(3)
                elif array[17] == "Stage IV":
                    stage_temp.append(4)
                elif array[17] == "Stage IIIB":
                    stage_temp.append(3)
                elif array[17] == "Stage IIA":
                    stage_temp.append(2)
                elif array[17] == "Stage IIC":
                    stage_temp.append(2)
                elif array[17] == "Stage IIB":
                    stage_temp.append(2)
                elif array[17] == "Stage IC":
                    stage_temp.append(1)
                else:
                    stage_temp.append(3)
                    
                if array[26] == "completeresponse":
                    cr_temp.append(1)
                else:
                    cr_temp.append(0)
                
                if array[26] == "partialresponse":
                    pr_temp.append(1)
                else:
                    pr_temp.append(0)
                
                if array[26] == "progressivedisease":
                    pd_temp.append(1)
                else:
                    pd_temp.append(0)
                
                if array[28] == "NA":
                    hrd_temp.append(-1)
                elif array[28] == "low":
                    hrd_temp.append(0)
                else:
                    hrd_temp.append(1)
                
                if array[25] == "NA":
                    brca_temp.append(-1)
                elif array[25] == "BRCAmut" or array[25] == "BRCA1meth":
                    brca_temp.append(1)
                else:
                    brca_temp.append(0)
                    
                if array[22] == "No residual disease R0":
                    debulking_temp.append(1)
                elif array[22] == "Residual disease R1":
                    debulking_temp.append(0)
                else:
                    debulking_temp.append(-1)
    print(len(filenames_temp))
    print(len(age_temp))
    print(len(stage_temp))
    print(len(debulking_temp))
    print(len(brca_temp))
    print(len(hrd_temp))
    print(len(pr_temp))
    print(len(cr_temp))
    print(len(pd_temp))
    print(len(os_days_temp))
    print(len(os_event_temp))
    print(len(pfs_days_temp))
    print(len(pfs_event_temp))

In [ ]:
# create output survival information for testing model
is_ovarian_cancer = 1
if is_ovarian_cancer:
    t_pfs_event_temp = []
    t_pfs_days_temp = []
    t_os_event_temp = []
    t_os_days_temp = []
    t_filenames_temp = []
    t_age_temp = []
    t_stage_temp = []
    t_hrd_temp = []
    t_cr_temp = []
    t_pr_temp = []
    t_pd_temp = []
    t_brca_temp = []
    t_debulking_temp = []
    flag = -1
    c = 0
    with open("survival_analysis_results/ovary/upmc.csv", newline='', encoding = "ISO-8859-1") as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                flag = 1
            else:
                array = row
                t_filenames_temp.append(array[0])
                t_age_temp.append(float(array[9]))
                
                if array[3] == "CR":
                    t_cr_temp.append(1)
                else:
                    t_cr_temp.append(0)
                    
                if array[3] == "PR":
                    t_pr_temp.append(1)
                else:
                    t_pr_temp.append(0)
                
                if array[3] == "PD":
                    t_pd_temp.append(1)
                else:
                    t_pd_temp.append(0)
                    
                if array[8] == "Yes":
                    t_debulking_temp.append(1)
                elif array[8] == "No":
                    t_debulking_temp.append(0)
                else:
                    t_debulking_temp.append(-1)
                    
                if array[12] == "0" and array[13] == "0":
                    t_brca_temp.append(0)
                else:
                    t_brca_temp.append(1)
                
                if array[1] == "1":
                    t_hrd_temp.append(1)
                elif array[1] == "0":
                    t_hrd_temp.append(0)
                else:
                    t_hrd_temp.append(-1)
                
                if array[4] == "DOD" or array[4] == "D-other":
                    t_os_event_temp.append(True)
                else:
                    t_os_event_temp.append(False)
                if array[7] == "" or array[7] == "NA":
                    t_os_days_temp.append(0)
                else:
                    t_os_days_temp.append(int(array[7])*30.5)
                    
                if array[5] == "Yes":
                    t_pfs_event_temp.append(True)
                else:
                    t_pfs_event_temp.append(False)
                if array[6] == "" or array[6] == "NA":
                    if array[7] == "" or array[7] == "NA":
                        t_pfs_days_temp.append(0)
                    else:
                        t_pfs_days_temp.append(int(array[7])*30.5)
                else:
                    t_pfs_days_temp.append(int(array[6])*30.5)
                
                if array[11] == "IIIc":
                    t_stage_temp.append(3)
                elif array[11] == "IV":
                    t_stage_temp.append(4)
                elif array[11] == "IVb":
                    t_stage_temp.append(4)
                elif array[11] == "IIIa":
                    t_stage_temp.append(3)
                elif array[11] == "IIIb":
                    t_stage_temp.append(3)
                elif array[11] == "IIIa1":
                    t_stage_temp.append(3)
                elif array[11] == "IIIa2":
                    t_stage_temp.append(3)
                elif array[11] == "II":
                    t_stage_temp.append(2)
                elif array[11] == "IIc":
                    t_stage_temp.append(2)
                elif array[11] == "IIb":
                    t_stage_temp.append(2)
                elif array[11] == "IIa":
                    t_stage_temp.append(2)
                elif array[11] == "I":
                    t_stage_temp.append(1)
                else:
                    t_stage_temp.append(3)
    print(len(t_filenames_temp))
    print(len(t_age_temp))
    print(len(t_stage_temp))
    print(len(t_debulking_temp))
    print(len(t_brca_temp))
    print(len(t_hrd_temp))
    print(len(t_pr_temp))
    print(len(t_cr_temp))
    print(len(t_pd_temp))
    print(len(t_os_days_temp))
    print(len(t_os_event_temp))
    print(len(t_pfs_days_temp))
    print(len(t_pfs_event_temp))

In [ ]:
# create output survival information for testing model
is_ovarian_cancer = 1
if is_ovarian_cancer:
    t1_pfs_event_temp = []
    t1_pfs_days_temp = []
    t1_os_event_temp = []
    t1_os_days_temp = []
    t1_filenames_temp = []
    t1_age_temp = []
    t1_stage_temp = []
    t1_hrd_temp = []
    t1_cr_temp = []
    t1_pr_temp = []
    t1_pd_temp = []
    t1_brca_temp = []
    t1_debulking_temp = []
    flag = -1
    c = 0
    with open("survival_analysis_results/ovary/ucla.csv", newline='', encoding = "ISO-8859-1") as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                flag = 1
                print(row)
            else:
                array = row
                t1_filenames_temp.append(array[3])
                t1_age_temp.append(float(array[4]))
                t1_os_days_temp.append(float(array[23])*30.5)
                
                if row[18] == "":
                    t1_pfs_days_temp.append(0.0)
                else:
                    t1_pfs_days_temp.append(float(array[18])*30.5)
                
                if array[25] == "1":
                    t1_os_event_temp.append(True)
                else:
                    t1_os_event_temp.append(False)
                
                if array[19] == "Yes":
                    t1_pfs_event_temp.append(True)
                else:
                    t1_pfs_event_temp.append(False)
                    
                if array[15] == "IIIC":
                    t1_stage_temp.append(3)
                elif array[15] == "IV":
                    t1_stage_temp.append(4)
                elif array[15] == "IIIB":
                    t1_stage_temp.append(3)
                elif array[15] == "IIIA":
                    t1_stage_temp.append(3)
                elif array[15] == "III":
                    t1_stage_temp.append(3)
                elif array[15] == "IIA":
                    t1_stage_temp.append(2)
                elif array[15] == "IIC":
                    t1_stage_temp.append(2)
                elif array[15] == "IIB":
                    t1_stage_temp.append(2)
                elif array[15] == "IC":
                    t1_stage_temp.append(1)
                else:
                    t1_stage_temp.append(3)
                    
    print(len(t1_filenames_temp))
    print(len(t1_age_temp))
    print(len(t1_stage_temp))
    print(len(t1_debulking_temp))
    print(len(t1_brca_temp))
    print(len(t1_hrd_temp))
    print(len(t1_pr_temp))
    print(len(t1_cr_temp))
    print(len(t1_pd_temp))
    print(len(t1_os_days_temp))
    print(len(t1_os_event_temp))
    print(len(t1_pfs_days_temp))
    print(len(t1_pfs_event_temp))

In [ ]:
tcga_features = []
for file in tcga_oc_files:
    filename = file.split("/")[-1]
    flag = -1
    file_features = []
    with open("survival_analysis_results/ovary/tcga/"+filename, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                array = row
                for index in range(0, len(row)):
                    val = float(row[index])
                    file_features.append(val)                
    tcga_features.append(file_features)
print(len(tcga_features))
print(tcga_features[0])

In [ ]:
upmc_features = []
for file in upmc_oc_files:
    filename = file.split("/")[-1]
    flag = -1
    file_features = []
    with open("survival_analysis_results/ovary/upmc/"+filename, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                for index in range(0, len(row)):
                    val = float(row[index])
                    file_features.append(val)
    upmc_features.append(file_features)
print(len(upmc_features))

In [ ]:
ucla_features = []
for file in ucla_oc_files:
    filename = file.split("/")[-1]
    flag = -1
    file_features = []
    with open("survival_analysis_results/ovary/ucla/"+filename, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                for index in range(0, len(row)):
                    val = float(row[index])
                    file_features.append(val)
    ucla_features.append(file_features)
print(len(ucla_features))

In [ ]:
train_pfs_event = []
train_pfs_days = []
train_os_event = []
train_os_days = []
train_age = []
train_stage = []
train_hrd = []
train_cr = []
train_pr = []
train_pd = []
train_brca = []
train_debulking = []
for index in range(0, len(tcga_oc_files)):
    filename1 = tcga_oc_files[index].split("/")[-1][:-4]
    flag = -1
    count = 0
    for index1 in range(0, len(filenames_temp)):
        filename2 = filenames_temp[index1]
        if filename1 == filename2:
            flag = 1
            train_pfs_event.append(pfs_event_temp[count])
            train_pfs_days.append(pfs_days_temp[count])
            train_os_event.append(os_event_temp[count])
            train_os_days.append(os_days_temp[count])
            train_age.append(age_temp[count])
            train_stage.append(stage_temp[count])
            train_hrd.append(hrd_temp[count])
            train_cr.append(cr_temp[count])
            train_pr.append(pr_temp[count])
            train_pd.append(pd_temp[count])
            train_brca.append(brca_temp[count])
            train_debulking.append(debulking_temp[count])
            break
        count += 1
print(len(train_age))
print(len(train_stage))
print(len(train_debulking))
print(len(train_brca))
print(len(train_hrd))
print(len(train_pr))
print(len(train_cr))
print(len(train_pd))
print(len(train_os_days))
print(len(train_os_event))
print(len(train_pfs_days))
print(len(train_pfs_event))

In [ ]:
test_pfs_event = []
test_pfs_days = []
test_os_event = []
test_os_days = []
test_age = []
test_stage = []
test_hrd = []
test_cr = []
test_pr = []
test_pd = []
test_brca = []
test_debulking = []
for index in range(0, len(upmc_oc_files)):
    filename1 = upmc_oc_files[index].split("/")[-1][:-4]
    flag = -1
    count = 0
    for index1 in range(0, len(t_filenames_temp)):
        filename2 = t_filenames_temp[index1]
        if filename1 == filename2:
            flag = 1
            test_pfs_event.append(t_pfs_event_temp[count])
            test_pfs_days.append(t_pfs_days_temp[count])
            test_os_event.append(t_os_event_temp[count])
            test_os_days.append(t_os_days_temp[count])
            test_age.append(t_age_temp[count])
            test_stage.append(t_stage_temp[count])
            test_hrd.append(t_hrd_temp[count])
            test_cr.append(t_cr_temp[count])
            test_pr.append(t_pr_temp[count])
            test_pd.append(t_pd_temp[count])
            test_brca.append(t_brca_temp[count])
            test_debulking.append(t_debulking_temp[count])
            break
        count += 1
    
    if flag == -1:
        print(filename1)
print(len(test_age))
print(len(test_stage))
print(len(test_debulking))
print(len(test_brca))
print(len(test_hrd))
print(len(test_pr))
print(len(test_cr))
print(len(test_pd))
print(len(test_os_days))
print(len(test_os_event))
print(len(test_pfs_days))
print(len(test_pfs_event))

In [ ]:
test_pfs_event_1 = []
test_pfs_days_1 = []
test_os_event_1 = []
test_os_days_1 = []
test_age_1 = []
test_stage_1 = []
for index in range(0, len(ucla_oc_files)):
    filename1 = ucla_oc_files[index].split("/")[-1][:-4]
    flag = -1
    count = 0
    for index1 in range(0, len(t1_filenames_temp)):
        filename2 = t1_filenames_temp[index1]
        if filename1 == filename2:
            flag = 1
            test_pfs_event_1.append(t1_pfs_event_temp[count])
            test_pfs_days_1.append(t1_pfs_days_temp[count])
            test_os_event_1.append(t1_os_event_temp[count])
            test_os_days_1.append(t1_os_days_temp[count])
            test_age_1.append(t1_age_temp[count])
            test_stage_1.append(t1_stage_temp[count])
            break
        count += 1
    
    if flag == -1:
        print(filename1)
print(len(test_age_1))
print(len(test_stage_1))
print(len(test_os_days_1))
print(len(test_os_event_1))
print(len(test_pfs_days_1))
print(len(test_pfs_event_1))

In [ ]:
train_features = np.array(tcga_features)
test_features = np.array(upmc_features+ucla_features)
train_censor = np.array(train_os_event)
train_days = np.array(train_os_days)
test_censor = np.array(test_os_event+test_os_event_1)
test_days = np.array(test_os_days+test_os_days_1)

In [ ]:
train_y = []
for index in range(0, len(train_censor)):
    train_y.append([train_censor[index], train_days[index]])
print(len(train_y))

test_y = []
for index in range(0, len(test_censor)):
    test_y.append([test_censor[index], test_days[index]])
print(len(test_y))

In [ ]:
# run on test set
group = []
train_group = []
features_train = train_features
features_test = test_features
y_train = train_y
dt = dtype=[('Status', '?'), ('Survival_in_days', '<f8')]
y_train = np.array([tuple(row) for row in y_train], dtype=dt)
scaler = MinMaxScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)
features_train_df = pd.DataFrame(features_train)
features_test_df = pd.DataFrame(features_test)

# fit model
estimator = CoxnetSurvivalAnalysis()
estimator.fit(features_train_df, y_train)
score, _, _, _, _ = concordance_index_censored(test_censor, test_days, estimator.predict(features_test_df))
#print("Test: " + str(score))
score, _, _, _, _ = concordance_index_censored(train_censor, train_days, estimator.predict(features_train_df))
#print("Train: " + str(score))

# get risk scores
train_risk_scores = estimator.predict(features_train_df)
test_risk_scores = estimator.predict(features_test_df)

In [ ]:
# train
median = np.median(train_risk_scores)
count_low = 0
count_high = 0
for index in range(0, len(train_risk_scores)):
    if train_risk_scores[index] > median:
        count_high += 1
        train_group.append(1)
    else:
        count_low += 1
        train_group.append(0)

# test
count_low = 0
count_high = 0
for index in range(0, len(test_risk_scores)):
    if test_risk_scores[index] > median:
        count_high += 1
        group.append(1)
    else:
        count_low += 1
        group.append(0)
print(median)

In [ ]:
print(*train_days, sep="; ")

In [ ]:
a = []
for index in range(0, len(train_censor)):
    if train_censor[index] == False:
        a.append(0)
    else:
        a.append(1)
print(*a, sep="; ")

In [ ]:
print(*train_risk_scores, sep='; ')

In [ ]:
print(*train_group, sep="; ")

In [ ]:
print(len(test_days))
print(*test_days, sep="; ")

In [ ]:
a = []
for index in range(0, len(test_censor)):
    if test_censor[index] == False:
        a.append(0)
    else:
        a.append(1)
print(len(a))
print(*a, sep="; ")

In [ ]:
print(*test_risk_scores, sep='; ')

In [ ]:
print(len(group))
print(*group, sep="; ")

In [ ]:
# find prognostic features from model trained above
count = 0
for index1 in range(0, len(estimator.coef_)):
    flag = -1
    for index2 in range(0, 100):
        if estimator.coef_[index1][index2] > 0 or estimator.coef_[index1][index2] < 0:
            flag = 1
            print(str(index1) + " " + str(estimator.coef_[index1][index2]))
            break
    if flag == 1:
        count += 1
print()
print("Prognostic features count = " + str(count))